<a href="https://colab.research.google.com/github/Tharun-web-glitch/PDFassistant/blob/main/PDFasst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf
!pip install sentence-transformers
!pip install faiss-cpu
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.2/494.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 85.0 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving example.pdf to example.pdf


In [ ]:
import fitz  # PyMuPDF

pdf_path = "example.pdf"

doc = fitz.open(pdf_path)
text = ""

for page in doc:
    text += page.get_text()

print(text[:1000])  # Preview first 1000 characters

 
• Table of
Contents
• Index
Cryptography and Network Security
Principles and Practices, Fourth Edition
By William Stallings
Publisher : Prentice Hall
Pub Date : November 16, 2005
Print ISBN-10 : 0-13-187316-4
Print ISBN-13 : 978-0-13-187316-2
eText ISBN-10 : 0-13-187319-9
eText ISBN-13 : 978-0-13-187319-3
Pages : 592
 
In this age of viruses
and hackers, of electronic eavesdropping and
electronic fraud, security is paramount.
As the disciplines of cryptography and network
security have matured, more practical, readily
available applications to enforce network security
have developed. This text provides a practical survey
of both the principles and practice of cryptography
and network security. First, the basic issues to be
addressed by a network security capability are
explored through a tutorial and survey of
cryptography and network security technology. Then,
the practice of network security is explored via
practical applications that have been implemented
and are in use today.
• T

In [ ]:
import re

def chunk_text(text, max_chunk_size=1000, overlap=200):
    sentences = re.split(r'(?<=[.!?]) +', text)

    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = current_chunk[-overlap:] + " " + sentence

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

chunks = chunk_text(text)
print("Total chunks:", len(chunks))

Total chunks: 2079


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-mpnet-base-v2')

embeddings = embedding_model.encode(chunks)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import faiss
import numpy as np

dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print("Stored in FAISS index")

Stored in FAISS index


In [ ]:
def retrieve(query, top_k=3):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [chunks[i] for i in indices[0]]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

gen_model_name = "google/flan-t5-large"

tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
generation_model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/558 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def generate_answer(context, question):
    prompt = f"""
You are an expert assistant.

Carefully read the context and answer the question completely.
If multiple pieces of information are relevant, combine them.
Do not copy incomplete fragments.Answer the question in clear bullet points.


Context:
{context}

Question:
{question}

Answer in clear complete sentences:
"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

    outputs = generation_model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.2,
        do_sample=False
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Ask Question
question = "What is cryptography?"

# 🔥 Retrieval step (ADD IT HERE)
retrieved_chunks = retrieve(question, top_k=6)

# Combine retrieved text
context = " ".join(retrieved_chunks)

# Generate answer
answer = generate_answer(context, question)

print("Answer:\n", answer)

Answer:
 cryptography The branch of cryptology dealing with the design of algorithms for encryption and decryption, intended to ensure the secrecy and/or authenticity of messages. Cryptology The study of secure communications, which encompasses both cryptography and cryptanalysis. decryption The translation of encrypted text or data (called ciphertext) into original text or data (called plaintext). Also called deciphering. differential cryptanalysis A technique in which chosen plaintexts with particular XOR difference patterns are encrypted.


In [ ]:
for i, chunk in enumerate(retrieved_chunks):
    print(f"\nChunk {i}:\n")
    print(chunk)


Chunk 0:

on, or forging encrypted information that will
be accepted as authentic.
cryptographic checksum
An authenticator that is a cryptographic function of both the
data to be authenticated and a secret key. Also referred to as a
message authentication code (MAC).
cryptography
The branch of cryptology dealing with the design of algorithms
for encryption and decryption, intended to ensure the secrecy
and/or authenticity of messages.
cryptology
The study of secure communications, which encompasses both
cryptography and cryptanalysis.
decryption
The translation of encrypted text or data (called ciphertext) into
original text or data (called plaintext). Also called deciphering.
differential cryptanalysis
A technique in which chosen plaintexts with particular XOR
difference patterns are encrypted.

Chunk 1:

Chapter 3 examines the most
widely used symmetric cipher: DES.
Before beginning, we define some terms. An original message is
known as the plaintext, while the coded message is call